In [6]:
from os import walk
import json
import re
from tqdm import tqdm
from utils import pmi, pmi_to_dict_adj_dict, docs_distribution, new_distribution, OptimizedIncrementalPMI
from scoring import compute_scores
import time
import pandas as pd
from textCleaning import get_file_names, extract_year_ipc
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import utils
from utils import docs_distribution, new_distribution, combine_columns, OptimizedIncrementalPMI
import copy
from novelty import ClusterKS

import importlib
importlib.reload(utils)
from utils import docs_distribution, new_distribution, combine_columns, OptimizedIncrementalPMI
import scoring
importlib.reload(scoring)
from scoring import compute_scores
import novelty
importlib.reload(novelty)
import surprise
importlib.reload(surprise)
from surprise import Surprise

In [7]:
import sys
import logging

nblog = open("nbMetrics_cB.log", "a+")
sys.stdout.echo = nblog
sys.stderr.echo = nblog

get_ipython().log.handlers[0].stream = nblog
get_ipython().log.setLevel(logging.INFO)

%autosave 5

Autosaving every 5 seconds


In [2]:
# w_size = 3
# path = "/home/edgarlanoue/data/csvCleaned/" #"C:/Users/edgar/OneDrive/Bureau/Ecole/HEC/A24/BrevetNLP/exemple données/cleaned/"
# pathOutput = "/home/edgarlanoue/metrics/" #"C:/Users/edgar/OneDrive/Bureau/Ecole/HEC/A24/BrevetNLP/exemple données/metrics/"
# # path = "C:/Users/edgar/OneDrive/Bureau/Ecole/HEC/A24/BrevetNLP/exemple données/cleaned/"
# # pathOutput = "C:/Users/edgar/OneDrive/Bureau/Ecole/HEC/A24/BrevetNLP/exemple données/metrics/"

# useClusters=True
# ipcList = ["H01L"]
# yearList = ["2016"]
# tE_cols = ["claims"]
# base_cols = ["claims", "background"]

In [8]:
def iterateMetrics(path, pathOutput, tE_cols, base_cols, w_size, useClusters, year, ipc):
    start_time = time.time()  # Record the starting time
    print(f"{year}")
    print(f"    {ipc}")
    tE = pd.read_csv(path+f"tE/{year}_{ipc}_tE_cleaned.csv")
    KS = pd.read_csv(path+f"KS/{year}_{ipc}_KS_cleaned.csv")
    ES = pd.read_csv(path+f"ES/{year}_{ipc}_ES_cleaned.csv")

    ### Transforming KB into distribution
    application_number = tE["application_number"]
    label = tE["label"]

    tE=combine_columns(tE, tE_cols) #[:10000]
    KS=combine_columns(KS, base_cols) #[:100000]
    ES=combine_columns(ES, base_cols) #[:100000]

    # Création pmi ES
    # current_time = time.time()  # Get the current time
    # elapsed_time = current_time - start_time
    # start_time = current_time
    # print(elapsed_time)

    print("ES")
    chunk_size = 10000  # Adjust based on available memory
    ES_PMI = OptimizedIncrementalPMI(3)

    for i in range(0, len(ES), chunk_size):
        # base_texts = (word for text in ES[i:i+chunk_size] for word in text.split())  # Generator
        base_texts = [word for text in (ES[i:i+chunk_size]) for word in text.split()]
        ES_PMI.update(base_texts)
        # print(i)

    instance_ES_pmi = ES_PMI.compute_pmi()
    base_bigram_set = set(instance_ES_pmi.keys())

    dict_ES_pmi = pmi_to_dict_adj_dict(instance_ES_pmi)



    # base_texts = [word for text in (ES) for word in text.split()]
    # ES_PMI = OptimizedIncrementalPMI(3)
    # ES_PMI.update(base_texts)
    # instance_ES_pmi = ES_PMI.compute_pmi()
    # base_bigram_set = set(instance_ES_pmi.keys())
    
    # dict_ES_pmi = pmi_to_dict_adj_dict(instance_ES_pmi)
    print("ES finidhsed")
    
    # current_time = time.time()  # Get the current time
    # elapsed_time = current_time - start_time
    # start_time = current_time
    # print(elapsed_time)

    print("KS")
    KS_matrix, KS_dist, KS_Count_matrix = docs_distribution(baseSpace=KS, tE=tE)
    KS_size = list(range(KS_matrix.shape[0]))
    print("KS finished")

    # current_time = time.time()  # Get the current time
    # elapsed_time = current_time - start_time
    # start_time = current_time
    # print(elapsed_time)

    # print("Cluster")
    if useClusters==True:
        KSClusterDiff1000 = ClusterKS(list_know_P=KS_matrix, new_Q= None, N=100, nbPtsPerCluster=1000)
        KSClusterDiff1000.clusterKS()
    # print("Cluster finished")    
    # ES_size = list(range(ES_matrix.shape[0]))
    
    # current_time = time.time()  # Get the current time
    # elapsed_time = current_time - start_time
    # start_time = current_time
    # print(elapsed_time)
    
    ## We set to 0 the distance here for each recipe -- difference needs to estimate distance between all points. 
    #This serves as optim to not calculate for each varaitions but only once since it is the same distance for all KB
    # print('train variations for recipe {} done'.format(recette), ' | train variation size : ', len(train_recettes))
    
    neighborhood_distance  = 0.4
    new_ratio_vec = []
    new_bin_vec = []
    uniq_ratio_vec = []
    uniq_bin_vec = []
    diff_ratio_vec = []
    diff_bin_vec = []
    neighborhood_distance_vec = []
    surpDiv_ratio_vec = []
    surpDiv_bin_vec = []
    # mean100_vec = []
    
    for i in range(len(tE)):
        # print("new KS dist (with 1 toEval)")
        select_variation = KS_size + [len(KS_size)+i]
        NewKS_dist, variation_dist = new_distribution(KS_Count_matrix, select_variation)
        
        # print("new ES pmi (with 1 toEval)")
        # # instance_ES_updated_pmi = copy.deepcopy(incremental_pmi_ES_only) # reprise du pmi du ES seulement
        # new_pmi_instance = OptimizedIncrementalPMI(window_size = w_size)
        # update_text = [word for text in tqdm(tE[i]) for word in text.split()] 
        # new_pmi_instance.update(update_text)
        # new_pmi = new_pmi_instance.compute_pmi()

        baseTexts_update = [tE[i]]
        update_text = [word for text in (baseTexts_update) for word in text.split()]
        new_pmi = OptimizedIncrementalPMI(window_size=3)
        new_pmi.update(update_text)
        newpmi_PMI = new_pmi.compute_pmi()
        
        # print("compute scores")
        if useClusters==True:
            results = compute_scores(KB_matrix=KS_matrix, KB_dist=KS_dist, NewKB_dist=NewKS_dist, variation_dist=variation_dist, 
                                    EB_PMI=instance_ES_pmi, base_bigram_set=base_bigram_set, dict_know_pmi=dict_ES_pmi, New_EB_PMI=newpmi_PMI,
                                    neighbor_dist=neighborhood_distance, useClusters=True, KSCluster=KSClusterDiff1000)
        else:
            results = compute_scores(KB_matrix=KS_matrix, KB_dist=KS_dist, NewKB_dist=NewKS_dist, variation_dist=variation_dist, 
                                    EB_PMI=instance_ES_pmi, base_bigram_set=base_bigram_set, dict_know_pmi=dict_ES_pmi, New_EB_PMI=newpmi_PMI,
                                    neighbor_dist=neighborhood_distance, useClusters=useClusters)

        new_ratio = results[0]
        new_bin = results[1]

        uniq_ratio = results[2]
        uniq_bin = results[3]

        diff_ratio = results[4]
        diff_bin = results[5]
        neighborhood_distance = results[6] #update neighborhood distance so it isn't reset
        # mean100 = results[7]

        # surpNew_ratio = results[7]
        # surpNew_bin = results[8]

        surpDiv_ratio = results[8]
        surpDiv_bin = results[9]

        new_ratio_vec.append(new_ratio)
        new_bin_vec.append(new_bin)

        uniq_ratio_vec.append(uniq_ratio)
        uniq_bin_vec.append(uniq_bin)

        diff_ratio_vec.append(diff_ratio)
        diff_bin_vec.append(diff_bin)
        neighborhood_distance_vec.append(neighborhood_distance)
        # mean100_vec.append(mean100)

        surpDiv_ratio_vec.append(surpDiv_ratio)
        surpDiv_bin_vec.append(surpDiv_bin)

        if i % 1000 == 0:
            current_time = time.time()  # Get the current time
            elapsed_time = current_time - start_time  # Calculate elapsed time
            if i % 1000 == 0:
                print(f"{i} on {len(tE)}, time since last print: {elapsed_time:.2f} seconds")
            start_time = current_time  # Reset the start time for the next interval
                # Reset the start time to measure time per iteration
            start_time = time.time()

    # print("df")

    
    start_time = current_time
    df = pd.DataFrame({
        "application_number": application_number,
        "label": label,

        "new_ratio": new_ratio_vec,
        "new_bin": new_bin_vec,

        "uniq_ratio": uniq_ratio_vec,
        "uniq_bin": uniq_bin_vec,

        "diff_ratio": diff_ratio_vec,
        "diff_bin": diff_bin_vec, 
        "neighboroud_distance": neighborhood_distance_vec,
        # "mean100": mean100_vec

        # "surpNew_ratio": surpNew_ratio,
        # "surpNew_bin": surpNew_bin,

        "surpDiv_ratio":surpDiv_ratio_vec,
        "surpDiv_bin": surpDiv_bin_vec
    })
    # current_time = time.time()  # Get the current time
    # elapsed_time = current_time - start_time
    # print(elapsed_time)

    tE_cols_str = "_".join(tE_cols)
    base_cols_str = "_".join(base_cols)
    df.to_csv(pathOutput + f'/{year}_{ipc}_{tE_cols_str}_vs_{base_cols_str}_Metrics.csv', index=False)   
    
    # del tE, KS, ES, KS_matrix, KS_dist, KS_Count_matrix, df
    # del new_ratio_vec, new_bin_vec, uniq_ratio_vec, uniq_bin_vec, diff_ratio_vec, diff_bin_vec, neighborhood_distance_vec
    # del surpDiv_ratio_vec, surpDiv_bin_vec, instance_ES_pmi, dict_ES_pmi, newpmi_PMI




def measureNov(path, pathOutput, tE_cols, base_cols, w_size, useClusters, yearList, ipcList):
    if yearList == [] or ipcList == []:
        tE_names = get_file_names(path+"tE")
        KS_names = get_file_names(path+"KS")
        ES_names = get_file_names(path+"ES")

        tE_set = set([extract_year_ipc(string) for string in tE_names])
        KS_set = set([extract_year_ipc(string) for string in KS_names])
        ES_set = set([extract_year_ipc(string) for string in ES_names])

        assert(tE_set==KS_set==ES_set)

        yearList = sorted(list(set([year for year, ipc in list(tE_set)])))
        ipcList = list(set([ipc for year, ipc in list(tE_set)]))
        print(yearList)
        print(ipcList)

    for year in yearList:
        for ipc in ipcList:
            iterateMetrics(path, pathOutput, tE_cols, base_cols, w_size, useClusters, year, ipc)
            

In [ ]:
w_size = 3
path = "/home/edgarlanoue/data/csvCleaned/" #"C:/Users/edgar/OneDrive/Bureau/Ecole/HEC/A24/BrevetNLP/exemple données/cleaned/"
pathOutput = "/home/edgarlanoue/metrics/metrics" #"C:/Users/edgar/OneDrive/Bureau/Ecole/HEC/A24/BrevetNLP/exemple données/metrics/"
# path = "C:/Users/edgar/OneDrive/Bureau/Ecole/HEC/A24/BrevetNLP/exemple données/cleaned/"
# pathOutput = "C:/Users/edgar/OneDrive/Bureau/Ecole/HEC/A24/BrevetNLP/exemple données/metrics/"

useClusters=True
ipcList = ["A61B"]
yearList = ["2012", "2013", "2014", "2015", "2016"]
tE_cols = ["claims"]
base_cols = ["claims", "background"]

measureNov(path, pathOutput, tE_cols, base_cols, w_size, useClusters, yearList, ipcList)

2012
    A61B
ES


In [ ]:
3

In [ ]:
# # start_time = time.time()  # Record the starting time
# # if yearList == [] or ipcList == []:
# #     tE_names = get_file_names(path+"tE")
# #     KS_names = get_file_names(path+"KS")
# #     ES_names = get_file_names(path+"ES")

# #     tE_set = set([extract_year_ipc(string) for string in tE_names])
# #     KS_set = set([extract_year_ipc(string) for string in KS_names])
# #     ES_set = set([extract_year_ipc(string) for string in ES_names])

# #     assert(tE_set==KS_set==ES_set)

# #     yearList = sorted(list(set([year for year, ipc in list(tE_set)])))
# #     ipcList = list(set([ipc for year, ipc in list(tE_set)]))
# #     print(yearList)
# #     print(ipcList)

# # for year in yearList:
# #     for ipc in ipcList:
# #         print(f"{year}")
# #         print(f"    {ipc}")
# #         tE = pd.read_csv(path+f"tE/{year}_{ipc}_tE_cleaned.csv")
# #         KS = pd.read_csv(path+f"KS/{year}_{ipc}_KS_cleaned.csv")
# #         ES = pd.read_csv(path+f"ES/{year}_{ipc}_ES_cleaned.csv")

# #         ### Transforming KB into distribution
# #         application_number = tE["application_number"]
# #         label = tE["label"]

# #         tE=combine_columns(tE, tE_cols) #[:10000]
# #         KS=combine_columns(KS, base_cols) #[:100000]
# #         ES=combine_columns(ES, base_cols) #[:100000]

# #         # Création pmi ES
# #         # current_time = time.time()  # Get the current time
# #         # elapsed_time = current_time - start_time
# #         # start_time = current_time
# #         # print(elapsed_time)

# #         print("ES")
# #         base_texts = [word for text in (ES) for word in text.split()]
# #         ES_PMI = OptimizedIncrementalPMI(3)
# #         ES_PMI.update(base_texts)
# #         instance_ES_pmi = ES_PMI.compute_pmi()
# #         base_bigram_set = set(instance_ES_pmi.keys())
        
# #         dict_ES_pmi = pmi_to_dict_adj_dict(instance_ES_pmi)
# #         print("ES finidhsed")
        
# #         # current_time = time.time()  # Get the current time
# #         # elapsed_time = current_time - start_time
# #         # start_time = current_time
# #         # print(elapsed_time)

# #         print("KS")
# #         KS_matrix, KS_dist, KS_Count_matrix = docs_distribution(baseSpace=KS, tE=tE)
# #         KS_size = list(range(KS_matrix.shape[0]))
# #         print("KS finished")

# #         # current_time = time.time()  # Get the current time
# #         # elapsed_time = current_time - start_time
# #         # start_time = current_time
# #         # print(elapsed_time)

# #         # print("Cluster")
# #         if useClusters==True:
# #             KSClusterDiff1000 = ClusterKS(list_know_P=KS_matrix, new_Q= None, N=100, nbPtsPerCluster=1000)
# #             KSClusterDiff1000.clusterKS()
# #         # print("Cluster finished")    
# #         # ES_size = list(range(ES_matrix.shape[0]))
        
# #         # current_time = time.time()  # Get the current time
# #         # elapsed_time = current_time - start_time
# #         # start_time = current_time
# #         # print(elapsed_time)
        
# #         ## We set to 0 the distance here for each recipe -- difference needs to estimate distance between all points. 
# #         #This serves as optim to not calculate for each varaitions but only once since it is the same distance for all KB
# #         # print('train variations for recipe {} done'.format(recette), ' | train variation size : ', len(train_recettes))
        
#         # neighborhood_distance  = 0.
# new_ratio_vec = []
# new_bin_vec = []
# uniq_ratio_vec = []
# uniq_bin_vec = []
# diff_ratio_vec = []
# diff_bin_vec = []
# neighborhood_distance_vec = []
# surpDiv_ratio_vec = []
# surpDiv_bin_vec = []
# # mean100_vec = []

# for i in tqdm(range(1227,len(tE))):
#     # print("new KS dist (with 1 toEval)")
#     select_variation = KS_size + [len(KS_size)+i]
#     NewKS_dist, variation_dist = new_distribution(KS_Count_matrix, select_variation)
    
#     # print("new ES pmi (with 1 toEval)")
#     # # instance_ES_updated_pmi = copy.deepcopy(incremental_pmi_ES_only) # reprise du pmi du ES seulement
#     # new_pmi_instance = OptimizedIncrementalPMI(window_size = w_size)
#     # update_text = [word for text in tqdm(tE[i]) for word in text.split()] 
#     # new_pmi_instance.update(update_text)
#     # new_pmi = new_pmi_instance.compute_pmi()

#     baseTexts_update = [tE[i]]
#     update_text = [word for text in (baseTexts_update) for word in text.split()]
#     new_pmi = OptimizedIncrementalPMI(window_size=3)
#     new_pmi.update(update_text)
#     newpmi_PMI = new_pmi.compute_pmi()
    
#     # print("compute scores")
#     if useClusters==True:
#         results = compute_scores(KB_matrix=KS_matrix, KB_dist=KS_dist, NewKB_dist=NewKS_dist, variation_dist=variation_dist, 
#                                 EB_PMI=instance_ES_pmi, base_bigram_set=base_bigram_set, dict_know_pmi=dict_ES_pmi, New_EB_PMI=newpmi_PMI,
#                                 neighbor_dist=neighborhood_distance, useClusters=True, KSCluster=KSClusterDiff1000)
#     else:
#         results = compute_scores(KB_matrix=KS_matrix, KB_dist=KS_dist, NewKB_dist=NewKS_dist, variation_dist=variation_dist, 
#                                 EB_PMI=instance_ES_pmi, base_bigram_set=base_bigram_set, dict_know_pmi=dict_ES_pmi, New_EB_PMI=newpmi_PMI,
#                                 neighbor_dist=neighborhood_distance, useClusters=useClusters)

#     new_ratio = results[0]
#     new_bin = results[1]

#     uniq_ratio = results[2]
#     uniq_bin = results[3]

#     diff_ratio = results[4]
#     diff_bin = results[5]
#     neighborhood_distance = results[6] #update neighborhood distance so it isn't reset
#     # mean100 = results[7]

#     # surpNew_ratio = results[7]
#     # surpNew_bin = results[8]

#     surpDiv_ratio = results[8]
#     surpDiv_bin = results[9]

#     new_ratio_vec.append(new_ratio)
#     new_bin_vec.append(new_bin)

#     uniq_ratio_vec.append(uniq_ratio)
#     uniq_bin_vec.append(uniq_bin)

#     diff_ratio_vec.append(diff_ratio)
#     diff_bin_vec.append(diff_bin)
#     neighborhood_distance_vec.append(neighborhood_distance)
#     # mean100_vec.append(mean100)

#     surpDiv_ratio_vec.append(surpDiv_ratio)
#     surpDiv_bin_vec.append(surpDiv_bin)

#     if i % 1000 == 0:
#         current_time = time.time()  # Get the current time
#         elapsed_time = current_time - start_time  # Calculate elapsed time
#         if i % 1000 == 0:
#             print(f"{i} on {len(tE)}, time since last print: {elapsed_time:.2f} seconds")
#         start_time = current_time  # Reset the start time for the next interval
#             # Reset the start time to measure time per iteration
#         start_time = time.time()

# # print("df")


# start_time = current_time
# df = pd.DataFrame({
#     "application_number": application_number,
#     "label": label,

#     "new_ratio": new_ratio_vec,
#     "new_bin": new_bin_vec,

#     "uniq_ratio": uniq_ratio_vec,
#     "uniq_bin": uniq_bin_vec,

#     "diff_ratio": diff_ratio_vec,
#     "diff_bin": diff_bin_vec, 
#     "neighboroud_distance": neighborhood_distance_vec,
#     # "mean100": mean100_vec

#     # "surpNew_ratio": surpNew_ratio,
#     # "surpNew_bin": surpNew_bin,

#     "surpDiv_ratio":surpDiv_ratio_vec,
#     "surpDiv_bin": surpDiv_bin_vec
# })
# # current_time = time.time()  # Get the current time
# # elapsed_time = current_time - start_time
# # print(elapsed_time)

# tE_cols_str = "_".join(tE_cols)
# base_cols_str = "_".join(base_cols)
# # df.to_csv(pathOutp

  0%|                                      | 4/17971 [00:14<17:06:34,  3.43s/it]

0.371462
0.357
0.36193
0.3609

75 minutes - 0.3577 -256, 63 min pour print
36 min - 0.3542 -128
28 min - fixed neighborhood

In [5]:
import psutil

# Get the current memory usage
memory_info = psutil.virtual_memory()

# Print memory usage in human-readable format
print(f"Total RAM: {memory_info.total / (1024 ** 3):.2f} GB")
print(f"Used RAM: {memory_info.used / (1024 ** 3):.2f} GB")
print(f"Free RAM: {memory_info.available / (1024 ** 3):.2f} GB")
print(f"Memory Usage: {memory_info.percent}%")

Total RAM: 31.42 GB
Used RAM: 3.07 GB
Free RAM: 27.86 GB
Memory Usage: 11.3%


In [8]:
df

,application_number,label,new_ratio,new_bin,uniq_ratio,uniq_bin,diff_ratio,diff_bin,neighboroud_distance,mean100
0,15227081,1,0.000397,0,0.513325,0,0.99,1,0.32,0.418779
1,15208121,1,0.000224,0,0.449711,0,0.00,0,0.32,0.230824
2,15194961,1,0.000483,0,0.571443,1,0.99,1,0.32,0.490976
3,15201349,1,0.000345,0,0.562358,1,0.99,1,0.32,0.420883
4,15292082,1,0.000178,0,0.584375,1,0.93,1,0.32,0.401762


### Fichier qui créé mes inputs (Proba dist for all, for each doc, and for new ones, and PMI calculations)

Si j'ai des textes, ça doit me ressortir des vecteurs de distributions pour mes documents KB, EB, et pour mes nouveaux et les combinaisons

## Pour chaque recette on compute la KB et la EB

thresholds: </br>

Novelty divergence threshold to know if a term significantly contribute (avg = 0.000923, std = 0.00161) -- set to : 0.0041 </br>
Novelty divergence threshold to know if a term significantly contribute in probability of appearing (avg = 14.640, std = 21.253) -- set to : 57.14 </br>

Novelty newness div threshold (avg = 9.787e-05, std = 0.0006) -- set to : 0.0014 </br>
Novelty newness prob threshold (avg = 0.000601, std = 0.00043) -- set to : 0.0014 </br>
Novelty uniqness dist threshold (avg = 0.341, std = 0.093) -- set to : 0.527 </br>
Novelty uniqness shift threshold (avg = 0.0755, std = 0.0270) -- set to : 0.1295 </br>
Novelty difference local threshold (avg = 0.0104, std = 0.0730) -- set to : 0.1564 </br>
Novelty difference global threshold (avg = 0.1099, std = 0.1539) -- set to : 0.4177 </br>
Novelty surprise new threshold (avg = 0.0016, std = 0.0052) -- set to : 0.0104 </br>
Novelty surprise dist threshold (avg = 0.00048, std = 0.00104) -- set to : 0.00256 </br>

In [ ]:
# ES[23900:23902]

In [11]:
# path = "C:/Users/edgar/OneDrive/Bureau/Ecole/HEC/A24/BrevetNLP/exemple données/cleaned/"
path = "/home/edgarlanoue/data/csvCleaned/"

In [12]:
tE_names = get_file_names(path+"tE")
KS_names = get_file_names(path+"KS")
ES_names = get_file_names(path+"ES")

tE_set = set([extract_year_ipc(string) for string in tE_names])
KS_set = set([extract_year_ipc(string) for string in KS_names])
ES_set = set([extract_year_ipc(string) for string in ES_names])

assert(tE_set==KS_set==ES_set)

yearList = [year for year, ipc in list(tE_set)]
ipcList = [ipc for year, ipc in list(tE_set)]

yearList = ["2012"]
ipcList = ["G06F"]

for year in yearList:
    for ipc in ipcList:
        print(f"{year}")
        print(f"    {ipc}")
        tE = pd.read_csv(path+f"tE/{year}_{ipc}_tE_cleaned.csv")
        KS = pd.read_csv(path+f"KS/{year}_{ipc}_KS_cleaned.csv")
        ES = pd.read_csv(path+f"ES/{year}_{ipc}_ES_cleaned.csv")

2012
    G06F


In [15]:
import importlib
import utils
importlib.reload(utils)
from utils import docs_distribution, new_distribution, combine_columns, OptimizedIncrementalPMI

In [16]:
application_number = tE["application_number"]
label = tE["label"]

In [17]:
columns = ['summary', 'abstract', 'background']

In [19]:
tE=combine_columns(tE, columns)
# KS=combine_columns(KS, ["claims"])
# ES=combine_columns(ES, ["claims"])

In [20]:
tE

0        aspect computer system employ dual channel mem...
1        object process device transmit receive datum a...
2         relate mobile terminal capable recommend wash...
3        brief description drawing complete understand ...
4        computer implement implement memory overlay pr...
                               ...                        
40955    touch responsive capacitive transparent substr...
40956    illustrative direct internet graphical user in...
40957    introduce selection representative concept sim...
40958    one system computer program manage memory host...
40959    one aspect server system control process game ...
Length: 40960, dtype: object

In [ ]:
# Prob_matrix, term_dist, Count_matrix = docs_distribution(baseSpace=KS, tE=tE)

In [ ]:
# print(sum(term_dist))
# print(Prob_matrix.sum())
# print(Count_matrix.sum())

0.9999999999999424
99087.0
45231752


In [ ]:
# KB_size = list(range(Prob_matrix.shape[0]))
# select_variation = KB_size + [len(KB_size)+1]

In [ ]:
# updated_Corpus_dist, Varations_dist = new_distribution(Count_matrix, select_variation)

In [ ]:
# print(sum(updated_Corpus_dist))
# print(sum(Varations_dist))
# #updated_Corpus_dist
# #Varations_dist

0.999999999999884
1.0


In [ ]:
# print(combine_columns(tE, ["claims", "background"])[0])
# print(combine_columns(tE, ["background"])[0])
# print(combine_columns(tE, ["claims"])[0])

In [ ]:
# base_texts = ' '.join(ES).split()
# print(base_texts[:10])

In [350]:
# ES_full.claims[122299]

In [ ]:
# indices = [i for i, text in enumerate(base_texts) if text == '0337']

In [113]:
# base_texts[indices[0]:indices[0]+5]

In [28]:
import importlib
import utils
importlib.reload(utils)
from utils import docs_distribution, new_distribution, combine_columns, OptimizedIncrementalPMI, pmi
from tqdm import tqdm

In [11]:
# base_texts = ' '.join(ES[:1000]).split()
base_texts = [word for text in tqdm(ES[:1000]) for word in text.split()]
# ES_pmi = pmi(base_texts, w_size=3)
# ES_pmi

100%|██████████| 1000/1000 [00:00<00:00, 12430.37it/s]


In [12]:
baseTexts_update = [tE[0]]
update_text = ' '.join(baseTexts_update).split()
new_pmi = OptimizedIncrementalPMI(window_size=3)
new_pmi.update(update_text)
print(new_pmi.bigram_counts)
print(new_pmi.word_counts)
print(new_pmi.total_words)

Counter({('dielectric', 'layer'): 45, ('layer', 'layer'): 42, ('layer', 'dielectric'): 40, ('encapsulation', 'layer'): 24, ('light', 'absorb'): 18, ('light', 'layer'): 18, ('absorb', 'layer'): 18, ('layer', 'comprise'): 17, ('layer', 'silicon'): 15, ('silicon', 'layer'): 15, ('diffusion', 'region'): 14, ('layer', 'open'): 12, ('nitride', 'layer'): 12, ('dielectric', 'dielectric'): 12, ('fourth', 'dielectric'): 12, ('fourth', 'layer'): 12, ('layer', 'encapsulation'): 11, ('comprise', 'layer'): 9, ('layer', 'monocrystalline'): 9, ('monocrystalline', 'semiconductor'): 9, ('monocrystalline', 'layer'): 9, ('semiconductor', 'layer'): 9, ('open', 'extend'): 9, ('extend', 'vertically'): 9, ('layer', 'light'): 9, ('comprise', 'silicon'): 9, ('additional', 'open'): 9, ('blanket', 'layer'): 9, ('layer', 'fourth'): 9, ('photodetector', 'comprise'): 8, ('comprise', 'dielectric'): 8, ('layer', 'photodetector'): 7, ('dielectric', 'monocrystalline'): 6, ('layer', 'semiconductor'): 6, ('open', 'vertica

In [ ]:
# # # Initialize the incremental PMI calculator
# instancePMI = IncrementalPMI(window_size=3)
# # incremental_pmi.update(base_texts)

In [13]:
instOptPMI = OptimizedIncrementalPMI(3)

In [ ]:
# # # Get the PMI scores after the first batch
# instancePMI.update(base_texts)

100%|██████████| 468114/468114 [00:00<00:00, 492092.07it/s]

totalWords =  468114
8042
247283


In [14]:
instOptPMI.update(base_texts)

In [15]:
dict_optPMI = instOptPMI.compute_pmi()

100%|██████████| 247283/247283 [00:00<00:00, 272587.83it/s]


In [29]:
from nltk.collocations import BigramAssocMeasures
bigram_measures = BigramAssocMeasures()

In [34]:
instOptPMI_2 = pmi(base_texts, 3)

In [35]:
instOptPMI_2

[(('1033', '1051'), 17.836500387165678),
 (('10h', '10i'), 17.836500387165678),
 (('10k', '10l'), 17.836500387165678),
 (('10n', '10o'), 17.836500387165678),
 (('13h', '11h'), 17.836500387165678),
 (('14e', '14f'), 17.836500387165678),
 (('1700', '2400'), 17.836500387165678),
 (('177', '262'), 17.836500387165678),
 (('18b', '18c'), 17.836500387165678),
 (('18c', '18d'), 17.836500387165678),
 (('18d', '18e'), 17.836500387165678),
 (('18e', '18f'), 17.836500387165678),
 (('18h', '18i'), 17.836500387165678),
 (('18i', '18j'), 17.836500387165678),
 (('18j', '18k'), 17.836500387165678),
 (('18k', '18l'), 17.836500387165678),
 (('18n', '18o'), 17.836500387165678),
 (('201', '162'), 17.836500387165678),
 (('565', '595'), 17.836500387165678),
 (('615', '680'), 17.836500387165678),
 (('6e', '6f'), 17.836500387165678),
 (('acetyl', 'cumyl'), 17.836500387165678),
 (('acetyl', 'pyrrolidinylmethyl'), 17.836500387165678),
 (('achievable', 'classical'), 17.836500387165678),
 (('agarose', 'alginic'), 

In [13]:
instOptPMI.total_words

136051420

In [14]:
print(instOptPMI.total_words)
print(dict_optPMI[3])

136051420
136051420


In [15]:
print(len(instOptPMI.word_counts))
print(len(dict_optPMI[1]))

84621
84621


In [16]:
word_counts_temp = dict_optPMI[1]
bigram_counts_temp = dict_optPMI[2]
total_words_temp = dict_optPMI[3]

In [17]:
bigram_counts_temp.update(new_pmi.bigram_counts)
word_counts_temp.update(new_pmi.word_counts)
total_words_temp_full = total_words_temp + new_pmi.total_words

In [18]:
temp_pmiInst = OptimizedIncrementalPMI(3, word_counts_temp, bigram_counts_temp, total_words_temp_full, False)

In [19]:
print(len(instOptPMI.word_counts))
print(len(temp_pmiInst.word_counts))

84621
84621


In [20]:
print(instOptPMI.total_words)
print(temp_pmiInst.total_words)

136051420
136052021


In [21]:
temp_pmi = temp_pmiInst.compute_pmi()

100%|██████████| 11075550/11075550 [06:54<00:00, 26732.97it/s] 


In [22]:
len([item[0][0] for item in temp_pmi[0]]) 

11075550

In [23]:
base_texts_set = set(base_texts)
update_text_set = set(update_text)

In [24]:
base_bigram_set = set(instOptPMI.bigram_counts.keys())

In [25]:
update_bigram_set = set(new_pmi.bigram_counts.keys())

In [26]:
dif_bigram_set = update_bigram_set-base_bigram_set
dif_word_set = update_text_set-base_texts_set

In [ ]:
# dict_bigram_new = {word: {key: value for key, value in new_pmi.bigram_counts.items() if word in key} for word in dif_bigram_set}
# dict_words_new = {key: value for key, value in new_pmi.word_counts.items() if key in dif_word_set}

In [ ]:
# bigrams_alreadyThere = {key for key in update_bigram_set if key in base_bigram_set}
# words_alreadyThere = {key for key in update_text_set if key in base_texts_set}
# print(len(words_alreadyThere))
# print(len(bigrams_alreadyThere))

57
106


In [27]:
import math

In [ ]:
pmi_base = dict_optPMI[0].copy()

dif_words_bigrams = {word for tup in dif_bigram_set for word in tup}
filtered_word_counts = {key: value for key, value in instOptPMI.word_counts.items() if key in dif_words_bigrams}
pmi_newPMI = new_pmi.compute_pmi()[0]

# inter_bigrams = {key: value for key, value in pmi_base.items() if key in new_pmi.bigram_counts.keys()}
# inter_words = {key: value for key, value in instOptPMI.word_counts.items() if key in update_text_set}
print("here")
new_bigrams = set(new_pmi.bigram_counts.keys())
filter_pmi = {key: value for key, value in pmi_base.items() if key in new_bigrams}
# Add missing keys from pmi_newPMI to filter_pmi with value [0, 0, 0, 0]
filter_pmi.update({key: [0, filtered_word_counts.get(key[0], 0), filtered_word_counts.get(key[1], 0), 0] for key in pmi_newPMI if key not in filter_pmi})
# dif_newPMI = {key: value for key, value in pmi_newPMI.items() if key in dif_bigram_set}
print("here2")
modified_new_values = {}
# modified_new_values.update(dif_newPMI)
# print(modified_new_values)
# Use dictionary comprehension to sum corresponding elements of the lists in both dictionaries
modified_new_values = {key: [a + b for a, b in zip(pmi_newPMI[key], filter_pmi[key])]
                 for key in pmi_newPMI}

print("here3")
# Update the first element of each list with the given formula
for key, value in modified_new_values.items():
    value[0] = math.log2(value[3] / (value[1] * value[2])) + math.log2((new_pmi.total_words + instOptPMI.total_words) * 0.5)
modified_new_values


print("here4")
for key, value in pmi_base.items():
    value[0] = value[0] - math.log2(instOptPMI.total_words) + math.log2(instOptPMI.total_words + new_pmi.total_words)

pmi_base.update(modified_new_values)

# vingatine de secondes

100%|██████████| 260/260 [00:00<?, ?it/s]


here
here2
here3
here4


In [ ]:
# baseTexts_update = [tE[0]]
# update_text = ' '.join(baseTexts_update).split()
# new_pmi = OptimizedIncrementalPMI(window_size=3)
# new_pmi.update(update_text)
# print(new_pmi.bigram_counts)
# print(new_pmi.word_counts)
# print(new_pmi.total_words)

Counter({('dielectric', 'layer'): 45, ('layer', 'layer'): 42, ('layer', 'dielectric'): 40, ('encapsulation', 'layer'): 24, ('light', 'absorb'): 18, ('light', 'layer'): 18, ('absorb', 'layer'): 18, ('layer', 'comprise'): 17, ('layer', 'silicon'): 15, ('silicon', 'layer'): 15, ('diffusion', 'region'): 14, ('layer', 'open'): 12, ('nitride', 'layer'): 12, ('dielectric', 'dielectric'): 12, ('fourth', 'dielectric'): 12, ('fourth', 'layer'): 12, ('layer', 'encapsulation'): 11, ('comprise', 'layer'): 9, ('layer', 'monocrystalline'): 9, ('monocrystalline', 'semiconductor'): 9, ('monocrystalline', 'layer'): 9, ('semiconductor', 'layer'): 9, ('open', 'extend'): 9, ('extend', 'vertically'): 9, ('layer', 'light'): 9, ('comprise', 'silicon'): 9, ('additional', 'open'): 9, ('blanket', 'layer'): 9, ('layer', 'fourth'): 9, ('photodetector', 'comprise'): 8, ('comprise', 'dielectric'): 8, ('layer', 'photodetector'): 7, ('dielectric', 'monocrystalline'): 6, ('layer', 'semiconductor'): 6, ('open', 'vertica

In [32]:
sorted(pmi_base)==sorted(temp_pmi[0])

True

In [41]:
sorted(pmi_base)==sorted(dict_optPMI[0])

False

In [ ]:
# dif_words_bigrams = {word for tup in dif_bigram_set for word in tup}
# filtered_word_counts = {key: value for key, value in instOptPMI.word_counts.items() if key in dif_words_bigrams}

In [ ]:
# pmi_newPMI = new_pmi.compute_pmi()[0]
# # inter_bigrams = {key: value for key, value in dict_optPMI[0].items() if key in new_pmi.bigram_counts.keys()}
# # inter_words = {key: value for key, value in instOptPMI.word_counts.items() if key in update_text_set}

# new_bigrams = set(new_pmi.bigram_counts.keys())
# filter_pmi = {key: value for key, value in dict_optPMI[0].items() if key in new_bigrams}
# # Add missing keys from pmi_newPMI to filter_pmi with value [0, 0, 0, 0]
# filter_pmi.update({key: [0, filtered_word_counts.get(key[0], 0), filtered_word_counts.get(key[1], 0), 0] for key in pmi_newPMI if key not in filter_pmi})
# # dif_newPMI = {key: value for key, value in pmi_newPMI.items() if key in dif_bigram_set}

100%|██████████| 260/260 [00:00<00:00, 473315.56it/s]


In [ ]:
# print(len(inter_bigrams))
# print(len(inter_words))
# print(len(inter_newPMI))
# print(len(dif_newPMI))
# print(len(inter_PMI))

106
57
106
154
106


In [38]:
sorted(filter_pmi.keys()) == sorted(pmi_newPMI.keys())

True

In [39]:
print(pmi_newPMI.get(('photodetector', 'comprise')))
print(dict_optPMI[0].get(('photodetector', 'comprise')))

[2.3857311297668105, 20, 23, 8]
None


In [111]:
print(pmi_newPMI.get(('photodetector', 'dielectric')))
print(filter_pmi.get(('photodetector', 'dielectric')))

[-0.17816975542651647, 20, 51, 3]
[0.20205851516731954, 7223, 49123, 6]


In [112]:
temp_pmi[0].get(('photodetector', 'dielectric'))

[0.20205214215184597, 7223, 49123, 6]

In [115]:
print(pmi_base.get(('photodetector', 'comprise')))
print(dict_optPMI[0].get(('photodetector', 'comprise')))

[0.748654828171226, 7243, 1900500, 340]
[0.748654828171226, 7243, 1900500, 340]


In [ ]:
# modified_new_values = {}
# # modified_new_values.update(dif_newPMI)
# # print(modified_new_values)
# # Use dictionary comprehension to sum corresponding elements of the lists in both dictionaries
# modified_new_values = {key: [a + b for a, b in zip(pmi_newPMI[key], filter_pmi[key])]
#                  for key in pmi_newPMI}

# # Update the first element of each list with the given formula
# for key, value in modified_new_values.items():
#     value[0] = math.log2(value[3] / (value[1] * value[2])) + math.log2((new_pmi.total_words + instOptPMI.total_words) * 0.5)
# modified_new_values

{('photodetector', 'comprise'): [2.7564133563944297, 20, 28, 8],
 ('photodetector', 'dielectric'): [0.4763054372016935, 20, 51, 3],
 ('comprise', 'dielectric'): [1.4059161093102963, 28, 51, 8],
 ('comprise', 'layer'): [0.20387233336565025, 28, 132, 9],
 ('dielectric', 'layer'): [1.6607300083391214, 51, 132, 45],
 ('dielectric', 'monocrystalline'): [2.213271031367901, 51, 12, 6],
 ('layer', 'monocrystalline'): [1.4262647547020988, 132, 12, 9],
 ('layer', 'semiconductor'): [1.2563397532597866, 132, 9, 6],
 ('monocrystalline', 'semiconductor'): [5.30073387261824, 12, 9, 9],
 ('monocrystalline', 'layer'): [1.4262647547020988, 12, 132, 9],
 ('semiconductor', 'layer'): [1.8413022539809427, 9, 132, 9],
 ('semiconductor', 'dielectric'): [1.6283085306467449, 9, 51, 3],
 ('layer', 'dielectric'): [1.4908050068968093, 132, 51, 40],
 ('layer', 'layer'): [0.18922555740124913, 132, 132, 42],
 ('dielectric', 'open'): [0.8913429364805383, 51, 15, 3],
 ('layer', 'open'): [1.51937415909358, 132, 15, 12],

In [ ]:
# for key, value in dict_optPMI[0].items():
#     value[0] = value[0] - math.log2(instOptPMI.total_words) + math.log2(instOptPMI.total_words + new_pmi.total_words)

In [49]:
dict_optPMI[0].update(modified_new_values)

In [51]:
temp_pmi[0].get(('thermal', 'cycling'))

[4.715771371897084, 18, 18, 18]

In [50]:
dict_optPMI[0].get(('thermal', 'cycling'))

[7.626259015244813, 18, 18, 18]

In [48]:
sorted(temp_pmi[0]) == sorted(dict_optPMI[0])

True

In [56]:
baseTexts_update = [tE[0]] #0s
update_text = [word for text in baseTexts_update for word in text.split()] #0s
instOptPMI.update(update_text) #0s
updated_pmi = instOptPMI.compute_pmi()

100%|██████████| 564/564 [00:00<00:00, 264016.46it/s]


In [58]:
instOptPMI.reset(update_text)
base_pmi_back = instOptPMI.compute_pmi()

100%|██████████| 564/564 [00:00<00:00, 315579.97it/s]


# Différence

In [9]:
import os

# Get the number of cores
num_cores = os.cpu_count()
print(f"Number of cores: {num_cores}")

import multiprocessing

# Get the number of cores
num_cores = multiprocessing.cpu_count()
print(f"Number of cores: {num_cores}")


Number of cores: 8
Number of cores: 8


In [14]:
import pickle

with open('KS_matrix.pkl', 'rb') as f:
    KS_matrix = pickle.load(f)
with open('variation_dist.pkl', 'rb') as f:
    variation_dist = pickle.load(f)

# Save an object
# with open('clusters.pkl', 'rb') as f:
#     clusters = pickle.load(f)

# with open('kmeans.pkl', 'rb') as f:
#     kmeans = pickle.load(f)

# with open('KSClusterDiff1000.pkl', 'rb') as f:
#     KSClusterDiff1000 = pickle.load(f)

In [7]:
KS_matrix, KS_dist, KS_Count_matrix = docs_distribution(baseSpace=KS, tE=tE)
KS_size = list(range(KS_matrix.shape[0]))
select_variation = KS_size + [len(KS_size)+1]
NewKS_dist, variation_dist = new_distribution(KS_Count_matrix, select_variation)

/home/edgarlanoue/brevetNLP/novelty/utils.py:255: RuntimeWarning: divide by zero encountered in divide
  Prob_KB_matrix = Old_matrix.multiply(1 / row_sums_reshaped)
/home/edgarlanoue/brevetNLP/novelty/utils.py:282: RuntimeWarning: divide by zero encountered in divide
  Variation_matrix = New_Count_matrix.multiply(1 / row_sums[:, None])


In [13]:
from novelty import Newness, Uniqueness, Difference
import novelty
import divergences
importlib.reload(novelty)
importlib.reload(divergences)
import tqdm
importlib.reload(tqdm)
from tqdm import tqdm
from divergences import Jensen_Shannon
from novelty import Newness, Uniqueness, Difference, ClusterKS
import random
import time
# instanceNew = Newness(known_P=term_dist, new_Q=Varations_dist)
# intanceUniq = Uniqueness(known_P=term_dist)

instanceDiff = Difference(list_know_P=KS_matrix, new_Q=variation_dist, N=100)
print((KS_matrix.shape))
print((variation_dist.shape))
print(type(KS_matrix))
print(type(print((variation_dist.shape))))

(20000, 20904)
(20904,)
<class 'scipy.sparse._csr.csr_matrix'>
(20904,)
<class 'NoneType'>


## Newness

In [14]:
instSurp = Newness(known_P=KS_dist, new_Q=variation_dist)
instSurp.divergent_terms()

(0.0, 0)

## Unique

In [15]:
instUniq = Uniqueness(known_P=KS_dist)
instUniq.dist_to_proto(variation_dist)

(0.4962168861441553, 1)

## Dist estimate

In [16]:
instanceDiff.dist_estimate(sample=True, sample_size=64, do_sample_P=True) # 4min

100%|██████████| 64/64 [03:53<00:00,  3.64s/it]


0.41533281305461756

In [19]:
KS_matrix

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 6551069 stored elements and shape (99087, 34772)>

In [22]:
KSClusterDiff1000 = ClusterKS(list_know_P=KS_matrix, new_Q=variation_dist, N=100, nbPtsPerCluster=1000)
clusters1000, kmeans1000 = KSClusterDiff1000.clusterKS()

In [24]:
KSClusterDiff1000.dist_estimate_clusters(128, nb_clusters= 4)

0.3626968934685003

In [ ]:
# neighdist = KSClusterDiff5000.mean_js_divergence_across_clusters_optimized(iterations=64) # 4min

In [ ]:
neighdist #4min

0.3937210817782374

In [14]:
# instanceDiff.ratio_to_neighbors(0.3)
# # 2 min (plus long maintenant...)

In [81]:
result_sansPara = instanceDiff.ratio_to_neighbors(0.3)

100%|██████████| 20000/20000 [00:15<00:00, 1286.15it/s]


In [82]:
result = instanceDiff.ratio_to_neighbors_joblib(0.3)

In [83]:
result_fC = instanceDiff.ratio_to_neighbors_fC(0.3)

20000it [00:42, 468.82it/s]


In [49]:
result_sansPara

(1.0, 1, 0.4008639076729114)

In [50]:
result[3]

0.4008639076729114

In [57]:
result_fC

(1.0, 1)

In [107]:
# columns = ["claims"]
# w_size = 3

# start_time = time.time()
# tE_names = get_file_names(path+"tE")
# KS_names = get_file_names(path+"KS")
# ES_names = get_file_names(path+"ES")

# tE_set = set([extract_year_ipc(string) for string in tE_names])
# KS_set = set([extract_year_ipc(string) for string in KS_names])
# ES_set = set([extract_year_ipc(string) for string in ES_names])

# assert(tE_set==KS_set==ES_set)

# yearList = [year for year, ipc in list(tE_set)]
# ipcList = [ipc for year, ipc in list(tE_set)]
# mean100 = []

# for year in yearList:
#     for ipc in ipcList:
#         print(f"{year}")
#         print(f"    {ipc}")
#         tE = pd.read_csv(path+f"tE/{year}_{ipc}_tE_cleaned.csv")
#         KS = pd.read_csv(path+f"KS/{year}_{ipc}_KS_cleaned.csv")
#         ES = pd.read_csv(path+f"ES/{year}_{ipc}_ES_cleaned.csv")

#         ### Transforming KB into distribution
#         application_number = tE["application_number"]
#         label = tE["label"]

#         tE=combine_columns(tE, columns)
#         KS=combine_columns(KS, columns)
#         ES=combine_columns(ES, columns)

#         KS_matrix,  KS_dist, KS_Count_matrix = docs_distribution(baseSpace=KS, tE=tE)
#         KS_size = list(range(KS_matrix.shape[0]))

In [ ]:
# for i in tqdm(range(30)): #len(tE))):
#     select_variation = KS_size + [len(KS_size)+i]
#     NewKS_dist, variation_dist = new_distribution(KS_Count_matrix, select_variation)
#     instanceDiff = Difference(list_know_P=KS_matrix, new_Q=variation_dist, N=100)
#     result = instanceDiff.ratio_to_neighbors_joblib(0.3)
#     results[i] = result[2]
#     mean100.append(np.mean(result[2]))

  3%|▎         | 1/30 [00:48<23:13, 48.05s/it]

[0.32828600895292104, 0.3367894737151954, 0.33735881666973533, 0.3377326761962186, 0.3377326761962186, 0.34994338349135157, 0.3500946449339537, 0.3504854566459765, 0.3521412952449994, 0.3549002325705106, 0.3573759877353144, 0.35848895117823454, 0.3600259256400127, 0.36077586974092535, 0.360838171577092, 0.3619303456942332, 0.36267587727621986, 0.36280054694077024, 0.3632271619422508, 0.36335639809179443, 0.36412293013256747, 0.3649999749020949, 0.3651507194427016, 0.36600778658089117, 0.36677968049148935, 0.3669571307894033, 0.3678760351403604, 0.36788189433718965, 0.3685031915856883, 0.3687936421087022, 0.3689400222893879, 0.36895950506651487, 0.3691044968854957, 0.3694515838670255, 0.36993591810110965, 0.36996143347136856, 0.3704990768311462, 0.370676556708124, 0.3710213751546242, 0.37105582019541805, 0.37186518905519644, 0.37212059453598334, 0.37237246898917137, 0.3724569340549698, 0.3727459871772099, 0.3733218020486572, 0.37404055089379984, 0.37433599988381705, 0.3744017455595744, 

  7%|▋         | 2/30 [01:40<23:43, 50.84s/it]

[0.24856490576553245, 0.2833943804195643, 0.2839789483248172, 0.28622493610984, 0.2872143139650787, 0.2909685522117347, 0.300012298722913, 0.30121893795509136, 0.30183697089704775, 0.30355800612665995, 0.3048660116986174, 0.30708348767655247, 0.30853920156083264, 0.30947971787318196, 0.3095941419645317, 0.31166120352718985, 0.31649936882082486, 0.3167553289742065, 0.320609485867105, 0.32076656101789985, 0.32219280757506896, 0.32293746654855227, 0.32551735187974795, 0.32602853091406103, 0.32736863164136987, 0.3282281951228, 0.3284025451369673, 0.32898122806992086, 0.3292765719080767, 0.32976869549849364, 0.3299683587434107, 0.33085484156464307, 0.33086605103671574, 0.33098691247896417, 0.33101993562087423, 0.33108988213431545, 0.3313535909178761, 0.3314424998513788, 0.33153493142247803, 0.33154545182384765, 0.3317319846373568, 0.3326194629824955, 0.3328468553029907, 0.33368495404202214, 0.3343239104158523, 0.33459560817540474, 0.3346835315794894, 0.3348155720813363, 0.33523286030021027,

 10%|█         | 3/30 [02:31<22:47, 50.63s/it]

...

[0.3196285986916808, 0.3245836582627296, 0.3256440134194319, 0.327855203198825, 0.32951816919009524, 0.33053598189169875, 0.33219137108137964, 0.3338100447793442, 0.33566929055046907, 0.33591958984892345, 0.33987425099696356, 0.3424769666815891, 0.3438638763924192, 0.3454664459036031, 0.34584654658264796, 0.3461820027929016, 0.3462948959736041, 0.34671339604045, 0.3468145747585451, 0.3469045132904759, 0.3470702431703304, 0.3484860842389422, 0.3487145361458943, 0.34897052404317047, 0.35011050451958653, 0.3506686624850254, 0.35180530420555534, 0.3522659500163475, 0.35271383119657973, 0.35288975384813914, 0.35366074797268554, 0.3540003804899225, 0.35444812824017435, 0.3551323108817592, 0.35546948830144587, 0.35560693243672936, 0.3556903247212722, 0.3560503474616076, 0.35606132001517204, 0.35616661467413996, 0.3564227613914835, 0.3566868959882009, 0.3569928438484452, 0.35819926782921535, 0.35941674942026225, 0.3597279977457476, 0.36038093298676255, 0.36080911260628484, 0.3613473722073943, 

 97%|█████████▋| 29/30 [26:26<00:53, 53.99s/it]

[0.18744078598339647, 0.22277691377585201, 0.2563244798329428, 0.25842047805919127, 0.2649242764348782, 0.2656972194146049, 0.2661256325572656, 0.2683966255922962, 0.2762533125851119, 0.2821320286701475, 0.28770009048374834, 0.2880259574475087, 0.28831763147052103, 0.2884946294563144, 0.28944212851740453, 0.28965129871252027, 0.2911990801321608, 0.29403027111103286, 0.294048113788543, 0.2941149494001314, 0.2947121783493276, 0.29488308298066523, 0.2954283816808957, 0.2955575278817476, 0.2957142504081808, 0.2971021487895996, 0.29772012729546005, 0.29934506127866334, 0.30104945323376275, 0.3017582832724217, 0.3019155667958462, 0.30228469735719826, 0.30279420907359766, 0.3033809033734654, 0.30398071952368255, 0.3045838535015243, 0.30481880819490875, 0.30536349465285595, 0.3055489781894063, 0.3072830629360767, 0.3075352899890194, 0.30761103818544455, 0.3077678354594907, 0.30783695419203927, 0.30783858566511746, 0.3078472264943076, 0.3083951743891771, 0.3094846155167908, 0.3097534189396546, 

100%|██████████| 30/30 [27:18<00:00, 54.61s/it]

[0.31753391454809454, 0.338225703843743, 0.34161975272257344, 0.35012506159741896, 0.35463511769333445, 0.3566973654365716, 0.35936998105500295, 0.3622957547519823, 0.36621887929234276, 0.3684229258097305, 0.3689546170824829, 0.3705140527883857, 0.372124194124326, 0.3722505350609873, 0.37251495577620775, 0.3729097809268315, 0.37728496924095806, 0.3777619189777752, 0.37776667127124797, 0.37871343161844123, 0.3788459640288458, 0.3792518246331571, 0.3793945082174528, 0.3802780165158663, 0.3808131608824983, 0.38134842002560077, 0.3814470710502749, 0.38147252071068316, 0.3821529959090969, 0.3822962847017227, 0.38334190453409467, 0.38484925567985295, 0.385222186670623, 0.38639258676131344, 0.3865969421626967, 0.3879716885241, 0.38880144066706457, 0.38882890462850217, 0.39026630362599485, 0.39026630362599485, 0.3903013573233669, 0.39073913924384346, 0.391030209045646, 0.39159393370625983, 0.39163079096890274, 0.3917312043217636, 0.3922999657048404, 0.3924358156397041, 0.39248565126506174, 0.3

In [ ]:
# df = pd.DataFrame(results)

# # Optionally rename columns to something more descriptive if needed
# df.columns = [f"Column_{col}" for col in df.columns]
# sorted_df = df.apply(sorted, axis=0)


In [ ]:
# sorted_df[:100].to_csv("C:/Users/edgar/OneDrive/Bureau/Ecole/HEC/A24/BrevetNLP/exemple données/metrics/dif_tests.csv")

In [31]:
dif100 = pd.read_csv("C:/Users/edgar/OneDrive/Bureau/Ecole/HEC/A24/BrevetNLP/exemple données/metrics/dif_tests/dif_tests.csv", index_col=0)
column_means_100 = np.array(dif100.mean(axis=0))
column_means_100

array([0.37087315, 0.33194159, 0.41241089, 0.40902273, 0.36181938,
       0.25764104, 0.45035365, 0.35171933, 0.40949922, 0.1923631 ,
       0.40943725, 0.36687228, 0.32833849, 0.4854088 , 0.27346766,
       0.27326013, 0.36136646, 0.38160361, 0.41403376, 0.29476125,
       0.38229775, 0.33432006, 0.41062786, 0.28682364, 0.38576328,
       0.32733492, 0.3703603 , 0.35780128, 0.30373527, 0.38759542])

In [164]:
dif100_dict = {i: dif100.iloc[:, i].tolist() for i in range(dif100.shape[1])}
dif100_dict

{0: [0.328286008952921,
  0.3367894737151954,
  0.3373588166697353,
  0.3377326761962186,
  0.3377326761962186,
  0.3499433834913515,
  0.3500946449339537,
  0.3504854566459765,
  0.3521412952449994,
  0.3549002325705106,
  0.3573759877353144,
  0.3584889511782345,
  0.3600259256400127,
  0.3607758697409253,
  0.360838171577092,
  0.3619303456942332,
  0.3626758772762198,
  0.3628005469407702,
  0.3632271619422508,
  0.3633563980917944,
  0.3641229301325674,
  0.3649999749020949,
  0.3651507194427016,
  0.3660077865808911,
  0.3667796804914893,
  0.3669571307894033,
  0.3678760351403604,
  0.3678818943371896,
  0.3685031915856883,
  0.3687936421087022,
  0.3689400222893879,
  0.3689595050665148,
  0.3691044968854957,
  0.3694515838670255,
  0.3699359181011096,
  0.3699614334713685,
  0.3704990768311462,
  0.370676556708124,
  0.3710213751546242,
  0.371055820195418,
  0.3718651890551964,
  0.3721205945359833,
  0.3723724689891713,
  0.3724569340549698,
  0.3727459871772099,
  0.3733218

In [32]:
import novelty
importlib.reload(novelty)
import novelty
from novelty import Difference
from novelty import ClusterKS

In [ ]:
# select_variation = KS_size + [len(KS_size)+1]
# NewKS_dist, variation_dist = new_distribution(KS_Count_matrix, select_variation)

In [43]:
KSClusterDiff800 = ClusterKS(list_know_P=KS_matrix, new_Q=variation_dist, N=100, nbPtsPerCluster=800)
clusters800, kmeans800 = KSClusterDiff800.clusterKS()

In [42]:
KSClusterDiff1000 = ClusterKS(list_know_P=KS_matrix, new_Q=variation_dist, N=100, nbPtsPerCluster=1000)
clusters1000, kmeans1000 = KSClusterDiff1000.clusterKS()

In [41]:
KSClusterDiff2000 = ClusterKS(list_know_P=KS_matrix, new_Q=variation_dist, N=100, nbPtsPerCluster=2000)
clusters2000, kmeans2000 = KSClusterDiff2000.clusterKS()

In [40]:
KSClusterDiff3000 = ClusterKS(list_know_P=KS_matrix, new_Q=variation_dist, N=100, nbPtsPerCluster=3000)
clusters3000, kmeans3000 = KSClusterDiff3000.clusterKS()

In [33]:
KSClusterDiff5000 = ClusterKS(list_know_P=KS_matrix, new_Q=variation_dist, N=100, nbPtsPerCluster=5000)
clusters5000, kmeans5000 = KSClusterDiff5000.clusterKS()

In [ ]:
result800_2 = KSClusterDiff800.ratio_to_neighbors_kmeans(variation_dist=variation_dist, nb_clusters=2) #1s

In [ ]:
result800_3 = KSClusterDiff800.ratio_to_neighbors_kmeans(variation_dist=variation_dist, nb_clusters=3) #1.5s

In [54]:
result800_4 = KSClusterDiff800.ratio_to_neighbors_kmeans(variation_dist=variation_dist, nb_clusters=4) #1.8s

In [ ]:
result1000_2 = KSClusterDiff1000.ratio_to_neighbors_kmeans(variation_dist=variation_dist, nb_clusters=2) #1s

In [53]:
result1000_3 = KSClusterDiff1000.ratio_to_neighbors_kmeans(variation_dist=variation_dist, nb_clusters=3) #1.7s

In [52]:
result1000_4 = KSClusterDiff1000.ratio_to_neighbors_kmeans(variation_dist=variation_dist, nb_clusters=4) #1.8s

In [ ]:
result2000_2 = KSClusterDiff2000.ratio_to_neighbors_kmeans(variation_dist=variation_dist, nb_clusters=2) #2s

In [47]:
result3000_2 = KSClusterDiff3000.ratio_to_neighbors_kmeans(variation_dist=variation_dist, nb_clusters=2) #3.1s

In [ ]:
result5000_1 = KSClusterDiff5000.ratio_to_neighbors_kmeans(variation_dist=variation_dist, nb_clusters=1) #2.2s

In [67]:
result5000_1[1]

0.36425617761858425

In [63]:
np.mean(result5000_1[0])

0.36425617761858425

In [70]:
results={}
# mean100 = {}
KS_matrix,  KS_dist, KS_Count_matrix = docs_distribution(baseSpace=KS, tE=tE)
KS_size = list(range(KS_matrix.shape[0]))

In [127]:
list = ["Réelle", "5000_1", "3000_2", "2000_2", "1000_4", "1000_3", "1000_2", "800_4", "800_3", "800_2"]

In [86]:

def KSCluster_rTN(KSCluster, nb_clusters):
    results={}
    for i in tqdm(range(30)): #len(tE))):
        select_variation = KS_size + [len(KS_size)+i]
        NewKS_dist, variation_dist = new_distribution(KS_Count_matrix, select_variation)
        instanceDiff = Difference(list_know_P=KS_matrix, new_Q=variation_dist, N=100)
        result = KSCluster.ratio_to_neighbors_kmeans(variation_dist=variation_dist, nb_clusters=nb_clusters) 
        results[i] = result[0]
    return results

In [87]:
results_5000_1 = KSCluster_rTN(KSClusterDiff5000, 1)

100%|██████████| 30/30 [02:43<00:00,  5.44s/it]


In [88]:
results_3000_2 = KSCluster_rTN(KSClusterDiff3000, 2)

100%|██████████| 30/30 [02:55<00:00,  5.84s/it]


In [89]:
results_2000_2 = KSCluster_rTN(KSClusterDiff2000, 2)

100%|██████████| 30/30 [02:14<00:00,  4.47s/it]


In [90]:
results_1000_4 = KSCluster_rTN(KSClusterDiff1000, 4)

100%|██████████| 30/30 [01:53<00:00,  3.77s/it]


In [91]:
results_1000_3 = KSCluster_rTN(KSClusterDiff1000, 3)

100%|██████████| 30/30 [01:33<00:00,  3.11s/it]


In [92]:
results_1000_2 = KSCluster_rTN(KSClusterDiff1000, 2)

100%|██████████| 30/30 [01:04<00:00,  2.13s/it]


In [93]:
results_800_4 = KSCluster_rTN(KSClusterDiff800, 4)

100%|██████████| 30/30 [01:41<00:00,  3.39s/it]


In [94]:
results_800_3 = KSCluster_rTN(KSClusterDiff800, 3)

100%|██████████| 30/30 [01:16<00:00,  2.56s/it]


In [95]:
results_800_2 = KSCluster_rTN(KSClusterDiff800, 2)

100%|██████████| 30/30 [00:57<00:00,  1.91s/it]


In [134]:
timeList = [5.44, 5.84, 4.47, 3.77, 3.11, 2.13, 3.39, 2.56, 1.91]

In [135]:
timeList

[5.44, 5.84, 4.47, 3.77, 3.11, 2.13, 3.39, 2.56, 1.91]

In [165]:
dict_list = [dif100_dict, results_5000_1, results_3000_2, results_2000_2, results_1000_4, results_1000_3, results_1000_2, results_800_4, results_800_3, results_800_2]

In [166]:
means_list = []

# Iterate over each key (0 to 29)
for key in range(30):
    # For each key, compute the mean for each dictionary in dict_list
    means_for_key = [np.mean(dict_item.get(key, [])) for dict_item in dict_list]
    means_list.append(means_for_key)

# Create a DataFrame from the list of means
df = pd.DataFrame(means_list, columns=[f'dict_{i+1}_mean' for i in range(len(dict_list))], index=[f'key_{key}' for key in range(30)])
df.columns=list

In [167]:
def mark_closest(row):
    # Compute the absolute differences between each column value and the first column value (excluding the first column itself)
    diffs = np.abs(row.iloc[1:].values - row.iloc[0])  # row.iloc[1:] excludes the first column
    # Find the index of the minimum difference
    closest_index = diffs.argmin() + 1  # Add 1 to the index to account for excluding the first column
    
    # Create a new row with the '*' added to the closest value (other than the first column)
    row_with_star = row.copy()  # Make a copy of the row to modify it
    row_with_star.iloc[closest_index] = str(row_with_star.iloc[closest_index]) + ' *'  # Add '*' next to closest value
    
    return row_with_star

# Apply the function to each row in the dataframe
df_with_stars = df.apply(mark_closest, axis=1)
df_with_stars

C:\Users\edgar\AppData\Local\Temp\ipykernel_32500\2654157700.py:9: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.3731068134599373 *' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  row_with_star.iloc[closest_index] = str(row_with_star.iloc[closest_index]) + ' *'  # Add '*' next to closest value
C:\Users\edgar\AppData\Local\Temp\ipykernel_32500\2654157700.py:9: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.33609949271888034 *' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  row_with_star.iloc[closest_index] = str(row_with_star.iloc[closest_index]) + ' *'  # Add '*' next to closest value
C:\Users\edgar\AppData\Local\Temp\ipykernel_32500\2654157700.py:9: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an

,Réelle,5000_1,3000_2,2000_2,1000_4,1000_3,1000_2,800_4,800_3,800_2
key_0,0.370873,0.3731068134599373 *,0.3761,0.379077,0.373512,0.375132,0.389017,0.374087,0.374202,0.375132
key_1,0.331942,0.364256,0.33609949271888034 *,0.349122,0.346239,0.355880,0.358501,0.346694,0.354748,0.355841
key_2,0.412411,0.423887,0.415911,0.417472,0.41548761207293566 *,0.415488,0.417443,0.418681,0.421245,0.423307
key_3,0.409023,0.441148,0.424606,0.426249,0.4195026722616945 *,0.424247,0.427685,0.419558,0.424573,0.427853
key_4,0.361819,0.399667,0.40574,0.382161,0.384241,0.384855,0.387910,0.3753881187728483 *,0.375388,0.380428
key_5,0.257641,0.26861,0.263024,0.263341,0.264294,0.264799,0.265681,0.26117266115619264 *,0.265267,0.265601
key_6,0.450354,0.476667,0.494815,0.484186,0.464644,0.483040,0.483259,0.4643044588010356 *,0.466435,0.470909
key_7,0.351719,0.375372,0.368518,0.37289,0.36339857955484056 *,0.371175,0.390933,0.365913,0.371042,0.376085
key_8,0.409499,0.435822,0.4241497963175977 *,0.434817,0.427538,0.435308,0.440403,0.435621,0.445281,0.455387
key_9,0.192363,0.199644,0.19456065433045963 *,0.195446,0.194693,0.195228,0.195374,0.195207,0.195265,0.196337


In [174]:
# Initialize a list to hold the counts of values greater than 0.42 for each key across all dictionaries
counts_list = []

# Iterate over each key (0 to 29)
for key in range(30):
    # For each key, count the number of values greater than 0.42 in each dictionary
    counts_for_key = [sum(1 for value in dict_item.get(key, []) if value > 0.43) for dict_item in dict_list]
    counts_list.append(counts_for_key)

# Create a DataFrame from the list of counts
df_counts = pd.DataFrame(counts_list, columns=[f'dict_{i+1}_count' for i in range(len(dict_list))], index=[f'key_{key}' for key in range(30)])
df_counts.columns=list
df_with_stars = df_counts.apply(mark_closest, axis=1)
df_with_stars

C:\Users\edgar\AppData\Local\Temp\ipykernel_32500\2654157700.py:9: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0 *' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  row_with_star.iloc[closest_index] = str(row_with_star.iloc[closest_index]) + ' *'  # Add '*' next to closest value
C:\Users\edgar\AppData\Local\Temp\ipykernel_32500\2654157700.py:9: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '27 *' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  row_with_star.iloc[closest_index] = str(row_with_star.iloc[closest_index]) + ' *'  # Add '*' next to closest value
C:\Users\edgar\AppData\Local\Temp\ipykernel_32500\2654157700.py:9: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. 

,Réelle,5000_1,3000_2,2000_2,1000_4,1000_3,1000_2,800_4,800_3,800_2
key_0,0,0 *,0,0,0,0,0,0,0,0
key_1,0,0 *,0,0,0,0,0,0,0,0
key_2,12,53,27 *,35,27,27,35,39,46,52
key_3,0,77,56,61,45 *,58,63,45,58,63
key_4,0,0 *,0,0,0,0,0,0,0,0
key_5,0,0 *,0,0,0,0,0,0,0,0
key_6,94,99 *,99,100,100,100,100,100,100,100
key_7,0,0 *,0,0,0,0,0,0,0,0
key_8,0,80,57 *,77,60,74,82,74,83,91
key_9,0,0 *,0,0,0,0,0,0,0,0


In [155]:
dict_list[0].get(1) == dict_list[9].get(1)

True

In [180]:
counts_list = []

# Iterate over each key (0 to 29)
for key in range(30):
    # Get the values of the first dictionary for this key (dict_1)
    first_dict_values = dict_list[0].get(key, [])
    
    # For each of the other dictionaries, count how many of their values for this key match any value in the first dictionary
    counts_for_key = []
    for dict_item in dict_list[1:]:  # Start from the second dictionary onward
        # Only compare if the key exists in both the first dictionary and the current dictionary
        if key in dict_item:
            # Count how many of the values for this key in the current dictionary appear in the first dictionary's values
            matching_values = sum(1 for value in dict_item.get(key, []) if value in first_dict_values)
            counts_for_key.append(matching_values)
        else:
            counts_for_key.append(0)  # If the key doesn't exist in the current dictionary, append 0
            
    counts_list.append(counts_for_key)

# Create a DataFrame from the list of counts
df_counts = pd.DataFrame(counts_list, columns=[f"dict_{i+2}_match_count" for i in range(len(dict_list) - 1)])

df_counts.columns = list[1:]
df_counts
def add_star_to_max(row):
    max_value = row.max()
    return row.apply(lambda x: f"{x}*" if x == max_value else x)

# Apply the function to each row
df_with_stars = df_counts.apply(add_star_to_max, axis=1)
df_with_stars

,5000_1,3000_2,2000_2,1000_4,1000_3,1000_2,800_4,800_3,800_2
0,50*,39,31,46,39,18,46,45,42
1,12,36*,17,18,12,10,23,16,14
2,31,47,45,48*,48*,43,41,34,30
3,11,24,20,25*,19,19,25*,19,19
4,11,1,20,14,14,14,34*,34*,28
5,21,33,35,28,28,28,38*,28,28
6,7,0,5,20*,6,6,19,14,10
7,18,32*,23,27,26,12,25,19,18
8,7,23*,10,15,7,6,10,5,4
9,23,29*,29*,29*,28,28,27,27,24


In [ ]:
# import pickle

# with open('KS_matrix.pkl', 'wb') as f:
#     pickle.dump(KS_matrix, f)
# with open('variation_dist.pkl', 'wb') as f:
#     pickle.dump(variation_dist, f)

# # Save an object
# with open('clusters.pkl', 'wb') as f:
#     pickle.dump(clusters, f)

# with open('kmeans.pkl', 'wb') as f:
#     pickle.dump(kmeans, f)

# with open('KSClusterDiff1000.pkl', 'wb') as f:
#     pickle.dump(KSClusterDiff1000, f)

PicklingError: Can't pickle <class 'novelty.ClusterKS'>: it's not the same object as novelty.ClusterKS

In [ ]:
# df = pd.concat([pd.DataFrame(column_means_100), pd.DataFrame(results10000_50[1])], axis=1)
# df.columns = ['Acutal', 'Kmean']
# df["difference"] = df["Kmean"]-df["Acutal"]
# meanSDActual = np.mean(df["Acutal"]) + np.std(df["Acutal"])
# meanSDActual
# meanSDKmean =  np.mean(df["Kmean"]) + np.std(df["Kmean"])
# (dif100 > meanSDActual).sum()
# (sorted_df > meanSDActual).sum()
# df = pd.concat([pd.DataFrame(column_means_100), pd.DataFrame((dif100 > meanSDActual).sum()).reset_index(drop=True), pd.DataFrame(results10000_50[1]), pd.DataFrame((sorted_df > meanSDActual).sum()).reset_index(drop=True)], axis=1)
# df.columns = ['Acutal', "Actual count > nD", 'Kmean', "Kmean count > nD"]
# print(meanSDActual)
# df


0.4170829135348558


,Acutal,Actual count > nD,Kmean,Kmean count > nD
0,0.370873,0,0.372398,0
1,0.331942,0,0.342854,0
2,0.412411,61,0.420026,69
3,0.409023,51,0.436080,85
4,0.361819,0,0.385135,0
5,0.257641,0,0.267409,0
6,0.450354,96,0.471068,99
7,0.351719,0,0.359013,0
8,0.409499,42,0.436029,85
9,0.192363,0,0.203135,0


In [ ]:
# df = pd.concat([pd.DataFrame(column_means_100), pd.DataFrame(results10000_50[1])], axis=1)
# df.columns = ['Acutal', 'Kmean']
# df["difference"] = df["Kmean"]-df["Acutal"]
# meanSDActual = np.mean(df["Acutal"]) + np.std(df["Acutal"])
# meanSDActual
# meanSDKmean =  np.mean(df["Kmean"]) + np.std(df["Kmean"])
# (dif100 > meanSDActual).sum()
# (sorted_df > meanSDActual).sum()
# df = pd.concat([pd.DataFrame(column_means_100), pd.DataFrame((dif100 > meanSDActual).sum()).reset_index(drop=True), pd.DataFrame(results10000_50[1]), pd.DataFrame((sorted_df > meanSDActual).sum()).reset_index(drop=True)], axis=1)
# df.columns = ['Acutal', "Actual count > nD", 'Kmean', "Kmean count > nD"]
# print(meanSDActual)
# df


0.4170829135348558


,Acutal,Actual count > nD,Kmean,Kmean count > nD
0,0.370873,0,0.372615,0
1,0.331942,0,0.338398,0
2,0.412411,61,0.420079,70
3,0.409023,51,0.436009,85
4,0.361819,0,0.383269,0
5,0.257641,0,0.269178,0
6,0.450354,96,0.464757,97
7,0.351719,0,0.363199,0
8,0.409499,42,0.438193,87
9,0.192363,0,0.202406,0


# Surprise

In [41]:
import importlib
import surprise
import divergences
importlib.reload(surprise)
importlib.reload(utils)
importlib.reload(divergences)
from surprise import Surprise
from utils import pmi_to_dict_adj, pmi_to_dict_adj_dict

In [42]:
instanceSurp = Surprise(modified_new_values)

In [43]:
instanceSurp.uniq_surprise_adj(pmi_to_dict_adj(filter_pmi), eps=0)

get_common_vectors


100%|██████████| 59/59 [00:00<?, ?it/s]


JSDiv


100%|██████████| 59/59 [00:00<00:00, 6951.23it/s]


(0.03425209947221691, 1)

In [44]:
# new_pmi_PMI = new_pmi.compute_pmi()[0]
instanceSurp2 = Surprise(temp_pmi[0])

In [47]:
dict_known = pmi_to_dict_adj_dict(dict_optPMI[0])

In [ ]:
instanceSurp2.uniq_surprise_adj(dict_known, eps=0) # extremement long...

get_common_vectors


  2%|▏         | 1880/84619 [01:41<1:14:17, 18.56it/s]


KeyboardInterrupt: 

TypeError: int() argument must be a string, a bytes-like object or a real number, not 'list'

In [92]:
291.9345918449097/14.629391401178028

19.95534768598793

In [86]:
38.107680224753366/8.151037970952764

4.675193559465042

In [73]:
27.55777515456278/59

0.467080934823098

In [72]:
5489.24742274444/8042

0.6825724226242775

In [66]:
sorted(modified_new_values) == sorted(filter_pmi)

True

In [62]:
dict_known = pmi_to_dict_adj_dict(dict_optPMI[0])

In [87]:
dict_known = pmi_to_dict_adj_dict(filter_pmi)
dict_new = pmi_to_dict_adj_dict(modified_new_values)

In [89]:
vecotr_tuples = instanceSurp.get_common_vectors_adj(dict_known, dict_new, epsilon = 0)

100%|██████████| 59/59 [00:00<00:00, 59088.81it/s]


In [ ]:
# vecotr_tuples.keys()

dict_keys([])

In [ ]:
# key_list[0]

'create'

In [90]:
vecotr_tuples.keys()

dict_keys(['pair', 'titanium', 'intrinsic', 'contact', 'concentration', 'open', 'carbide', 'vertically', 'depth', 'cover', 'additional', 'atom', 'encapsulation', 'comprise', 'region', 'top', 'material', 'nitride', 'approximately', 'peak', 'bottom', 'locate', 'interlayer', 'oxide', 'blanket', 'halfway', 'germanium', 'adjacent', 'diffusion', 'cm3', 'extend', 'dopant', 'stack', 'one', 'photodetector', 'position', 'least', 'layer', 'light', 'fill', 'silicon', 'laterally', 'equal', 'profile', 'fourth', 'polysilicon', 'tantalum', 'surface', 'pit', 'monocrystalline', 'diode', 'immediately', 'dielectric', 'thicknesse', 'semiconductor', 'multiple', 'conformal', 'amount', 'absorb'])

In [91]:
key_list = list(vecotr_tuples.keys())
for i in tqdm(range(len(key_list))):
    tuple_known = vecotr_tuples[key_list[i]][0]
    tuple_new = vecotr_tuples[key_list[i]][1]
    if sum(tuple_known) != 0 and sum(tuple_new) != 0:   #### We can't compare to non existing vectors neither
            instanceSurp.JS.JSDiv(tuple_known, tuple_new)

100%|██████████| 59/59 [00:00<00:00, 2503.73it/s]


In [ ]:
# for entry in (key_list):
#     tuple_known = vecotr_tuples[entry][0]
#     tuple_new = vecotr_tuples[entry][1]
    
#     #### We want only positive PMI score -- no negative values for not going nan or inf values
#     tuple_known = [max(0., val) for val in tuple_known]
#     tuple_new = [max(0., val) for val in tuple_new]   
#     if sum(tuple_known) != 0 and sum(tuple_new) != 0:   #### We can't compare to non existing vectors neither
#         surprise_dists.append(self.JS.JSDiv(tuple_known, tuple_new))